In [1]:
# Create GAN for independent and identically distributed data

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import secrets

import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import math
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

import keras
from keras import layers
from keras import ops


class DataGenerator:
    def __init__(self, seq_len=32, latent_dim=25):
        # self.P = np.random.uniform(0, 1, (6, 6))
        # print(self.P, np.sum(self.P, axis=1, keepdims=True))
        # self.P = self.P / np.sum(self.P, axis=1, keepdims=True)
        self.states = [0, 1]
        self.actions = [0, 1, 2]
        self.latent_dim = latent_dim
        self.seq_len = seq_len
        # print(self.P)
        # self.build_generator()
        # self.build_discriminator()
        # self.build_gan()
        # print(self.P)

    def generate_real_samples(self, n_samples):
        X_total = []
        labels = []
        for i in range(n_samples):
            current_state = 0
            policy = np.array(self.select_policy())
            labels.append(policy)
            X = []
            while len(X) < self.seq_len:
                if current_state == 0:
                    action = np.random.choice([0, 1], p=policy)
                    if action == 0:
                        next_state = np.random.choice([0, 1])
                        if next_state == 0:
                            reward = 5
                        else:
                            reward = 5
                    else:
                        next_state = 1
                        reward = 10
                else:
                    action = 2
                    next_state = 1
                    reward = -1
                current_state_cat = keras.utils.to_categorical(
                    current_state, num_classes=2
                )
                action_cat = keras.utils.to_categorical(action, num_classes=3)
                reward = (reward + 1) / 11
                X.append(np.concatenate([current_state_cat, action_cat, [reward]]))
                current_state = next_state

            X = np.array(X)
            # X = tf.keras.utils.to_categorical(X, num_classes=6)
            X_total.append(X)
        X = np.array(X_total)
        y = np.ones((n_samples, 1)) * 0.9
        # print(X.shape, y.shape)
        # labels = tf.keras.utils.to_categorical(labels, num_classes=4)
        labels = np.array(labels)
        return X

    def select_policy(self):
        # if np.random.uniform() < 0.1:
        #     if np.random.uniform() <= 0.5:
        #         policy = [1.0, 0.0]
        #     else:
        #         policy = [0.0, 1.0]
        # else:
        #     policy = np.random.uniform(0, 1, 2)
        #     policy = policy / np.sum(policy)
        policy = np.array([0.7, 0.3])
        return policy
    def show_episode(self, x):
        x1 = np.argmax(x[:, :2], axis=1)
        x2 = np.argmax(x[:, 2:5], axis=1)
        x3 = x[:, 5]
        for i in range(self.seq_len):
            print(
                "State:", x1[i], "Action:", x2[i], "Reward:", np.rint((x3[i] * 11) - 1))

    def calculate_final_P(self, X):
        
        X1 = np.argmax(X[:, :, :2], axis=2)
        X2 = np.argmax(X[:, :, 2:5], axis=2)
        X3 = X[:, :, 5]
        # X1 = np.argmax(X1, axis=1)
        # X2 = np.argmax(X2, axis=1)
        print("Probabilitity of starting position being 0:", np.mean(X1[:, 0] == 0))
        print("Probabilitity of starting position being 1:", np.mean(X1[:, 0] == 1))

        print(
            "Probability of action 0 being taken in state 0:", np.mean(X2[X1 == 0] == 0)
        )
        print(
            "Probability of action 1 being taken in state 0:", np.mean(X2[X1 == 0] == 1)
        )
        print(
            "Probability of action 2 being taken in state 1:", np.mean(X2[X1 == 1] == 2)
        )
        ps = np.zeros((2, 3, 2))
        for i in range(n_samples):
            for j in range(self.seq_len - 1):
                ps[X1[i, j], X2[i, j], X1[i, j + 1]] += 1
        ps = ps / np.sum(ps, axis=2, keepdims=True)
        print(ps)


datagen = DataGenerator()
data = datagen.generate_real_samples(100000)
data2 = datagen.generate_real_samples(100000)

2024-04-29 09:03:17.451668: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 09:03:20.350125: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/TensorRT-8.4.0.6//lib:/usr/local/cuda-11.6/lib64:/usr/local/apps/python-3.10.2/lib:/usr/local/apps/cuDNN/8.4.0-cuda-11.6/lib
2024-04-29 09:03:20.350285: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 

In [2]:
data = tf.data.Dataset.from_tensor_slices(data)
data2 = tf.data.Dataset.from_tensor_slices(data2)

2024-04-29 09:05:25.060915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 09:05:25.664121: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9626 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:02:00.0, compute capability: 7.5


In [3]:
# data

num_epochs = 1  # train for at least 50 epochs for good results
image_size = 32


# sampling
min_signal_rate = 0.02
max_signal_rate = 0.95

# architecture
embedding_dims = 32
embedding_max_frequency = 1000.0
widths = [32, 64, 96, 128]
# widths = [64, 128, 256, 512]
has_attention = [False, False, True, True]
block_depth = 2

# optimization
batch_size = 64
ema = 0.999
learning_rate = 1e-3
weight_decay = 1e-4

In [4]:

def preprocess_image(data):
    # center crop image
    # height = ops.shape(data["image"])[0]
    # width = ops.shape(data["image"])[1]
    # crop_size = ops.minimum(height, width)
    # image = tf.image.crop_to_bounding_box(
    #     data["image"],
    #     (height - crop_size) // 2,
    #     (width - crop_size) // 2,
    #     crop_size,
    #     crop_size,
    # )

    # resize and clip
    # for image downsampling it is important to turn on antialiasing
    # image = tf.image.grayscale_to_rgb(data["image"])
    # image = tf.image.resize(image, size=[image_size, image_size], antialias=True)
    return ops.clip(data, 0.0, 1.0)


def prepare_dataset(dataset_name):
    # the validation dataset is shuffled as well, because data order matters
    # for the KID estimation
    return (
        dataset_name
        .map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        .cache()
        .repeat(1)
        .shuffle(10 * batch_size)
        .batch(batch_size, drop_remainder=True)
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )


# load dataset
train_dataset = prepare_dataset(data)
val_dataset = prepare_dataset(data)

In [8]:
@keras.saving.register_keras_serializable()
def sinusoidal_embedding(x):
    embedding_min_frequency = 1.0
    frequencies = ops.exp(
        ops.linspace(
            ops.log(embedding_min_frequency),
            ops.log(embedding_max_frequency),
            embedding_dims // 2,
        )
    )
    angular_speeds = ops.cast(2.0 * math.pi * frequencies, "float32")
    embeddings = ops.concatenate(
        [ops.sin(angular_speeds * x), ops.cos(angular_speeds * x)], axis=2
    )
    return embeddings


def ResidualBlock(width):
    def apply(x, temb):
        input_width = x.shape[2]
        if input_width == width:
            residual = x
        else:
            residual = layers.Conv1D(width, kernel_size=1)(x)
        x = layers.BatchNormalization(center=False, scale=False)(x)
        x = layers.Conv1D(width, kernel_size=3, padding="same", activation="swish")(x)
        temb = layers.Dense(width, activation="swish")(temb)
        x = layers.Add()([x, temb])
        x = layers.Conv1D(width, kernel_size=3, padding="same")(x)
        x = layers.Add()([x, residual])
        return x

    return apply


def DownBlock(width, block_depth):
    def apply(x, temb):
        x, skips = x
        for _ in range(block_depth):
            x = ResidualBlock(width)(x, temb)
            skips.append(x)
        # x = layers.AveragePooling1D(pool_size=2)(x)
        x = layers.Conv1D(width, kernel_size=3, strides=2, padding="same")(x)
        return x

    return apply


def UpBlock(width, block_depth):
    def apply(x, temb):
        x, skips = x
        x = layers.UpSampling1D(size=2)(x)
        x = layers.Conv1D(width, kernel_size=3, padding="same")(x)
        for _ in range(block_depth):
            x = layers.Concatenate()([x, skips.pop()])
            x = ResidualBlock(width)(x, temb)
        return x

    return apply


def get_network(image_size, widths, block_depth):
    noisy_images = keras.Input(shape=(image_size, 6))
    noise_variances = keras.Input(shape=(1, 1))
    # labels = keras.Input(shape=(2,))
    # print("DASDA")
    e = layers.Lambda(sinusoidal_embedding, output_shape=(1, 32))(noise_variances)
    # l = layers.Embedding(2, 32)(labels)
    
    # print(e.shape,l.shape)
    temb = e
    e = layers.UpSampling1D(size=image_size)(temb)
    
    x = layers.Conv1D(widths[0], kernel_size=1)(noisy_images)
    x = layers.Concatenate()([x, e])

    skips = []
    for i, width in enumerate(widths[:-1]):
        x = DownBlock(width, block_depth)([x, skips], temb)
        # if has_attention[i]:
        #     x = layers.MultiHeadAttention(
        #         num_heads=10, key_dim=width
        #     )(x, x)


    for i in range(block_depth):
        x = ResidualBlock(widths[-1])(x, temb)
        # if i != block_depth - 1:
        #     x = layers.MultiHeadAttention(
        #         num_heads=10, key_dim=widths[-1] 
        #     )(x, x)

    for i, width in enumerate(reversed(widths[:-1])):
        x = UpBlock(width, block_depth)([x, skips], temb)
        # if has_attention[-i - 1]:
        #     x = layers.MultiHeadAttention(
        #         num_heads=10, key_dim=width
        #     )(x, x)

    x = layers.Conv1D(6, kernel_size=1, kernel_initializer="zeros")(x)
    # x2 = layers.Conv1D(3, kernel_size=1, kernel_initializer="zeros", activation='softmax')(x)
    # x3 = layers.Conv1D(1, kernel_size=1, kernel_initializer="zeros", activation='sigmoid')(x)
    # x = layers.Concatenate()([x1,x2,x3])

    return keras.Model([noisy_images, noise_variances], x, name="residual_unet")

In [9]:
@keras.saving.register_keras_serializable()
class DiffusionModel(keras.Model):
    def __init__(self, image_size, widths, block_depth):
        super().__init__()

        self.normalizer = layers.Normalization()
        self.network = get_network(image_size, widths, block_depth)
        self.ema_network = keras.models.clone_model(self.network)

    def compile(self, **kwargs):
        super().compile(**kwargs)

        self.noise_loss_tracker = keras.metrics.Mean(name="n_loss")
        self.image_loss_tracker = keras.metrics.Mean(name="i_loss")

    @property
    def metrics(self):
        return [self.noise_loss_tracker, self.image_loss_tracker]

    def denormalize(self, images):
        # convert the pixel values back to 0-1 range
        images = self.normalizer.mean + images * self.normalizer.variance**0.5
        return ops.clip(images, 0.0, 1.0)

    def diffusion_schedule(self, diffusion_times):
        # diffusion times -> angles
        start_angle = ops.cast(ops.arccos(max_signal_rate), "float32")
        end_angle = ops.cast(ops.arccos(min_signal_rate), "float32")

        diffusion_angles = start_angle + diffusion_times * (end_angle - start_angle)

        # angles -> signal and noise rates
        signal_rates = ops.cos(diffusion_angles)
        noise_rates = ops.sin(diffusion_angles)
        # note that their squared sum is always: sin^2(x) + cos^2(x) = 1

        return noise_rates, signal_rates

    def denoise(self, noisy_images, noise_rates, signal_rates, training):
        # the exponential moving average weights are used at evaluation
        if training:
            network = self.network
        else:
            network = self.ema_network

        # predict noise component and calculate the image component using it
        pred_noises = network([noisy_images, noise_rates**2], training=training)
        pred_images = (noisy_images - noise_rates * pred_noises) / signal_rates

        return pred_noises, pred_images

    def reverse_diffusion(self, initial_noise, diffusion_steps):
        # reverse diffusion = sampling
        num_images = initial_noise.shape[0]
        step_size = 1.0 / diffusion_steps

        # important line:
        # at the first sampling step, the "noisy image" is pure noise
        # but its signal rate is assumed to be nonzero (min_signal_rate)
        next_noisy_images = initial_noise
        for step in range(diffusion_steps):
            noisy_images = next_noisy_images

            # separate the current noisy image to its components
            diffusion_times = ops.ones((num_images, 1, 1)) - step * step_size
            noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=False
            )
            # network used in eval mode

            # remix the predicted components using the next signal and noise rates
            next_diffusion_times = diffusion_times - step_size
            next_noise_rates, next_signal_rates = self.diffusion_schedule(
                next_diffusion_times
            )
            next_noisy_images = (
                next_signal_rates * pred_images + next_noise_rates * pred_noises
            )
            # this new noisy image will be used in the next step

        return pred_images

    def generate(self, num_images, diffusion_steps):
        # noise -> images -> denormalized images
        initial_noise = keras.random.normal(
            shape=(num_images, image_size, 6)
        )
        generated_images = self.reverse_diffusion(initial_noise, diffusion_steps)
        generated_images = self.denormalize(generated_images)
        return generated_images

    def train_step(self, images):
        # normalize images to have standard deviation of 1, like the noises
        images = self.normalizer(images, training=True)
        noises = keras.random.normal(shape=(batch_size, image_size, 6))

        # sample uniform random diffusion times
        diffusion_times = keras.random.uniform(
            shape=(batch_size, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # mix the images with noises accordingly
        noisy_images = signal_rates * images + noise_rates * noises

        with tf.GradientTape() as tape:
            # train the network to separate noisy images to their components
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=True
            )

            noise_loss = self.loss(noises, pred_noises)  # used for training
            image_loss = self.loss(images, pred_images)  # only used as metric

        gradients = tape.gradient(noise_loss, self.network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.network.trainable_weights))

        self.noise_loss_tracker.update_state(noise_loss)
        self.image_loss_tracker.update_state(image_loss)

        # track the exponential moving averages of weights
        for weight, ema_weight in zip(self.network.weights, self.ema_network.weights):
            ema_weight.assign(ema * ema_weight + (1 - ema) * weight)

        # KID is not measured during the training phase for computational efficiency
        return {m.name: m.result() for m in self.metrics[:-1]}

    def test_step(self, images):
        # normalize images to have standard deviation of 1, like the noises
        images = self.normalizer(images, training=False)
        noises = keras.random.normal(shape=(batch_size, image_size, 6))

        # sample uniform random diffusion times
        diffusion_times = keras.random.uniform(
            shape=(batch_size, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # mix the images with noises accordingly
        noisy_images = signal_rates * images + noise_rates * noises

        # use the network to separate noisy images to their components
        pred_noises, pred_images = self.denoise(
            noisy_images, noise_rates, signal_rates, training=False
        )

        noise_loss = self.loss(noises, pred_noises)
        image_loss = self.loss(images, pred_images)

        self.image_loss_tracker.update_state(image_loss)
        self.noise_loss_tracker.update_state(noise_loss)

        # measure KID between real and generated images
        # this is computationally demanding, kid_diffusion_steps has to be small
        # images = self.denormalize(images)
        # generated_images = self.generate(
        #     num_images=batch_size, diffusion_steps=kid_diffusion_steps
        # )
        # self.kid.update_state(images, generated_images)

        return {m.name: m.result() for m in self.metrics}

    def plot_images(self, epoch=None, logs=None, num_rows=1, num_cols=1):
        # plot random generated images for visual evaluation of generation quality
        generated_images = self.generate(
            num_images=num_rows * num_cols,
            diffusion_steps=20,
        )

        # print(generated_images)

In [10]:
# create and compile the model
model = DiffusionModel(image_size, widths, block_depth)
# below tensorflow 2.9:
# pip install tensorflow_addons
# import tensorflow_addons as tfa
# optimizer=tfa.optimizers.AdamW
model.compile(
    optimizer=keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    ),
    loss=keras.losses.mean_absolute_error,
)
# pixelwise mean absolute error is used as loss

# save the best model based on the validation KID metric
checkpoint_path = "checkpoints/diffusion_model.weights.h5"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor="val_i_loss",
    # mode="min",
    save_best_only=True,
)

# calculate mean and variance of training dataset for normalization
model.normalizer.adapt(train_dataset)

# run training and plot generated images periodically
# model.fit(
#     train_dataset,
#     epochs=100,
#     validation_data=val_dataset,
#     callbacks=[
#         # keras.callbacks.LambdaCallback(on_epoch_end=model.plot_images),
#         # checkpoint_callback,
#     ],
# )

In [11]:
model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[
        # keras.callbacks.LambdaCallback(on_epoch_end=model.plot_images),
        # checkpoint_callback,
    ],
)

Epoch 1/10


2024-04-29 09:07:54.211505: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x560fe46da0d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-29 09:07:54.211589: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-04-29 09:07:54.753024: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 09:07:57.653671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8400


   1/1562 ━━━━━━━━━━━━━━━━━━━━ 16:28:44 38s/step - n_loss: 0.8070

2024-04-29 09:08:15.325498: I tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:325] ptxas warning : Registers are spilled to local memory in function 'fusion_1140', 728 bytes spill stores, 608 bytes spill loads

2024-04-29 09:08:15.452816: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 69s 20ms/step - n_loss: 0.1587 - i_loss: 0.2268 - val_i_loss: 1.7201 - val_n_loss: 0.5885
Epoch 2/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - n_loss: 0.0478 - i_loss: 0.1074 - val_i_loss: 0.5139 - val_n_loss: 0.1915
Epoch 3/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 27s 17ms/step - n_loss: 0.0369 - i_loss: 0.0820 - val_i_loss: 0.1362 - val_n_loss: 0.0519
Epoch 4/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 29s 18ms/step - n_loss: 0.0313 - i_loss: 0.0709 - val_i_loss: 0.0563 - val_n_loss: 0.0214
Epoch 5/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - n_loss: 0.0284 - i_loss: 0.0654 - val_i_loss: 0.0406 - val_n_loss: 0.0153
Epoch 6/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 27s 17ms/step - n_loss: 0.0263 - i_loss: 0.0618 - val_i_loss: 0.0364 - val_n_loss: 0.0135
Epoch 7/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - n_loss: 0.0245 - i_loss: 0.0578 - val_i_loss: 0.0332 - val_n_loss: 0.0121
Epoch 8/10
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - n_loss: 0.0240 - i_loss: 0.05

In [12]:
# load the best model and generate images
# model.load_weights(checkpoint_path)
x = (model.generate(10000,20))

In [13]:
datagen.show_episode(x[4])

State: 0 Action: 1 Reward: 10.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1 Action: 2 Reward: -1.0
State: 1

In [14]:
def calculate_final_P(X):
        
    X1 = np.argmax(X[:, :, :2], axis=2)
    X2 = np.argmax(X[:, :, 2:5], axis=2)
    X3 = X[:, :, 5]
    # X1 = np.argmax(X1, axis=1)
    # X2 = np.argmax(X2, axis=1)
    print("Probabilitity of starting position being 0:", np.mean(X1[:, 0] == 0))
    print("Probabilitity of starting position being 1:", np.mean(X1[:, 0] == 1))

    print(
        "Probability of action 0 being taken in state 0:", np.mean(X2[X1 == 0] == 0)
    )
    print(
        "Probability of action 1 being taken in state 0:", np.mean(X2[X1 == 0] == 1)
    )
    print(
        "Probability of action 2 being taken in state 1:", np.mean(X2[X1 == 1] == 2)
    )
    ps = np.zeros((2, 3, 2))
    for i in range(x.shape[0]):
        for j in range(32 - 1):
            ps[X1[i, j], X2[i, j], X1[i, j + 1]] += 1
    ps = ps / np.sum(ps, axis=2, keepdims=True)
    print(ps)
calculate_final_P(x)

Probabilitity of starting position being 0: 1.0
Probabilitity of starting position being 1: 0.0
Probability of action 0 being taken in state 0: 0.7763393584106822
Probability of action 1 being taken in state 0: 0.2236606415893177
Probability of action 2 being taken in state 1: 1.0
[[[0.23932879 0.76067121]
  [0.         1.        ]
  [       nan        nan]]

 [[       nan        nan]
  [       nan        nan]
  [0.         1.        ]]]


/tmp/ipykernel_11503/2276824880.py:24: RuntimeWarning: invalid value encountered in divide
  ps = ps / np.sum(ps, axis=2, keepdims=True)


In [22]:
# print(np.array([i for i in train_dataset]).shape)
calculate_final_P(datagen.generate_real_samples(100000))

Probabilitity of starting position being 0: 1.0
Probabilitity of starting position being 1: 0.0
Probability of action 0 being taken in state 0: 0.6995637100229831
Probability of action 1 being taken in state 0: 0.3004362899770169
Probability of action 2 being taken in state 1: 1.0
[[[0.4992624 0.5007376]
  [0.        1.       ]
  [      nan       nan]]

 [[      nan       nan]
  [      nan       nan]
  [0.        1.       ]]]


/tmp/ipykernel_9951/2276824880.py:24: RuntimeWarning: invalid value encountered in divide
  ps = ps / np.sum(ps, axis=2, keepdims=True)


In [ ]:
pred_data = model.generate(2000, 20)
i = 0
for d in pred_data:
    if tuple(np.rint(d.reshape(-1)).astype(int)) in nset:
        i += 1
        print(i)
print(i)

In [34]:
model.network.summary()

Model: "residual_unet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 1, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_10      │ (None, 32, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_5 (Lambda)   │ (None, 1, 32)     │          0 │ input_layer_11[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_200 (Conv1D) │ (None, 32, 64)    │        448 │ input_layer_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling1d_20    │ (None, 32, 32)    │          0 │ lambda_5[0][0]    │
│ (UpSampling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_35      │ (None, 32, 96)    │          0 │ conv1d_200[0][0], │
│ (Concatenate)       │                   │            │ up_sampling1d_20… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 96)    │        192 │ concatenate_35[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_202 (Conv1D) │ (None, 32, 64)    │     18,496 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_203 (Conv1D) │ (None, 32, 64)    │     12,352 │ conv1d_202[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_201 (Conv1D) │ (None, 32, 64)    │      6,208 │ concatenate_35[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_70 (Add)        │ (None, 32, 64)    │          0 │ conv1d_203[0][0], │
│                     │                   │            │ conv1d_201[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 64)    │        128 │ add_70[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_204 (Conv1D) │ (None, 32, 64)    │     12,352 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_205 (Conv1D) │ (None, 32, 64)    │     12,352 │ conv1d_204[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_71 (Add)        │ (None, 32, 64)    │          0 │ conv1d_205[0][0], │
│                     │                   │            │ add_70[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d_… │ (None, 16, 64)    │          0 │ add_71[0][0]      │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 64)    │        128 │ average_pooling1… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_207 (Conv1D) │ (None, 16, 128)   │     24,704 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_208 (Conv1D) │ (None, 16, 128)   │     49,280 │ conv1d_207[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_206 (Conv1D) │ (None, 16, 128)   │      8,320 │ average_pooling1

 Total params: 6,098,374 (23.26 MB)

 Trainable params: 6,090,886 (23.23 MB)

 Non-trainable params: 7,488 (29.25 KB)